In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = None
def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def search_amazon(product):
    global driver
    try:
        search_bar = driver.find_element(By.ID, "twotabsearchtextbox")
        search_bar.send_keys(product)
        search_bar.send_keys(Keys.RETURN)
        time.sleep(2)
        products = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")
        for product in products:
            title = product.find_element(By.CSS_SELECTOR, "span.a-text-normal").text
            price = product.find_element(By.CSS_SELECTOR, "span.a-price").text
            print("Title:", title)
            print("Price:", price)
            print("=" * 50)
    except Exception as e:
        print("Error:", e)

def main():
    try:
        user_input = input("Enter the product you want to search on Amazon India: ")
        init_driver("https://images.google.com/")
        if driver:
            search_amazon(user_input)
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Enter the product you want to search on Amazon India: guitar
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="twotabsearchtextbox"]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF78AB37012+3522402]
	(No symbol) [0x00007FF78A758352]
	(No symbol) [0x00007FF78A605ABB]
	(No symbol) [0x00007FF78A64BF0E]
	(No symbol) [0x00007FF78A64C08C]
	(No symbol) [0x00007FF78A68E437]
	(No symbol) [0x00007FF78A66F09F]
	(No symbol) [0x00007FF78A68BDA3]
	(No symbol) [0x00007FF78A66EE03]
	(No symbol) [0x00007FF78A63F4D4]
	(No symbol) [0x00007FF78A6405F1]
	GetHandleVerifier [0x00007FF78AB69B9D+3730157]
	GetHandleVerifier [0x00007FF78ABBF02D+4079485]
	GetHandleVerifier [0x00007FF78ABB75D3+4048163]
	GetHandleVerifier [0x00007FF78A88A649+718233]
	(No symbol) [0x00007FF78A764A3F]
	(N

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
driver = None

def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def scrape_product_details(product):
    global driver
    try:
        search_bar = driver.find_element(By.ID, "twotabsearchtextbox")
        search_bar.clear()
        search_bar.send_keys(product)
        search_bar.send_keys(Keys.RETURN)
        time.sleep(2)
        
        product_details = []

        # Loop through the first 3 pages (or less if fewer pages are available)
        for page in range(3):
            products = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")
            for product in products:
                try:
                    product_link = product.find_element(By.CSS_SELECTOR, "h2 a")
                    product_link.send_keys(Keys.CONTROL + Keys.RETURN)  # Open link in new tab
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))  # Wait until new tab opens
                    driver.switch_to.window(driver.window_handles[1])  # Switch to the new tab
                    time.sleep(2)

                    details = {}
                    try:
                        details["Product URL"] = driver.current_url
                    except:
                        details["Product URL"] = "-"

                    try:
                        details["Brand Name"] = driver.find_element(By.CSS_SELECTOR, "span.a-size-base.po-break-word").text
                    except:
                        details["Brand Name"] = "-"

                    try:
                        details["Name of the Product"] = driver.find_element(By.ID, "productTitle").text.strip()
                    except:
                        details["Name of the Product"] = "-"

                    try:
                        details["Price"] = driver.find_element(By.CSS_SELECTOR, "span.a-price").text.split("\n")[0]
                    except:
                        details["Price"] = "-"

                    try:
                        details["Return/Exchange"] = driver.find_element(By.CSS_SELECTOR, "#icon-farm-container span").text
                    except:
                        details["Return/Exchange"] = "-"

                    try:
                        details["Expected Delivery"] = driver.find_element(By.CSS_SELECTOR, "span['data-csa-c-delivery-time']").text
                    except:
                        details["Expected Delivery"] = "-"

                    try:
                        details["Availability"] = driver.find_element(By.ID, "availability").text.strip()
                    except:
                        details["Availability"] = "-"

                    product_details.append(details)

                    driver.close()  # Close the tab
                    driver.switch_to.window(driver.window_handles[0])  # Switch back to the original tab
                except Exception as e:
                    print("Error processing product:", e)

            # Navigate to the next page if available
            try:
                next_page = driver.find_element(By.PARTIAL_LINK_TEXT, "Next")
                next_page.click()
                time.sleep(2)
            except:
                break  # Break if next page button is not found

        return product_details

    except Exception as e:
        print("Error:", e)
        return []

def save_to_csv(product_details):
    df = pd.DataFrame(product_details)
    df.to_csv("amazon_products.csv", index=False)

def main():
    try:
        user_input = input("Enter the product you want to search on Amazon India: ")
        init_driver("https://www.amazon.in")
        if driver:
            product_details = scrape_product_details(user_input)
            save_to_csv(product_details)
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Enter the product you want to search on Amazon India: Guitar
Error processing product: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF78AB37012+3522402]
	(No symbol) [0x00007FF78A758352]
	(No symbol) [0x00007FF78A605ABB]
	(No symbol) [0x00007FF78A5F1574]
	(No symbol) [0x00007FF78A5F1E3F]
	(No symbol) [0x00007FF78A606045]
	(No symbol) [0x00007FF78A5E232F]
	(No symbol) [0x00007FF78A67C401]
	(No symbol) [0x00007FF78A66F020]
	(No symbol) [0x00007FF78A63F4D4]
	(No symbol) [0x00007FF78A6405F1]
	GetHandleVerifier [0x00007FF78AB69B9D+3730157]
	GetHandleVerifier [0x00007FF78ABBF02D+4079485]
	GetHandleVerifier [0x00007FF78ABB75D3+4048163]
	GetHandleVerifier [0x00007FF78A88A649+718233]
	(No symbol) [0x00007FF78A764A3F]
	(No symbol) [0x00007FF78A75FA94]
	(No symbol) [0x00007FF78A75FBC2]
	(No symbol) [0x00007FF78A74F2E4]
	BaseThreadInit

In [3]:
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import shutil
import os

driver = None
def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def create_clean_folder(folder_path):
    try:
        # Check if the folder exists
        if os.path.exists(folder_path):
            # If the folder exists, remove it and its contents
            shutil.rmtree(folder_path)
        
        # Create the folder
        os.makedirs(folder_path)
        
        print(f"Folder '{folder_path}' created.")
    except Exception as e:
        print(f"Error creating folder: {e}")

def extract_images(count=10):
    global driver
    image_urls = []
    try:
        images = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")
        for index, image in enumerate(images[:count], start=1):
            image_url = image.get_attribute("src")
            if image_url:
                image_urls.append((index, image_url))
        return image_urls
    except Exception as e:
        print("Error:", e)
        return []

import base64

def save_base64_image(base64_string, filename):
    # Split the base64 string to get the image data
    _, base64_data = base64_string.split(',', 1)
    image_type = re.search(r"data:image/(\w+);base64,", base64_string).group(1)
    try:
        # Decode the base64 data into bytes
        image_data = base64.b64decode(base64_data)
        # Write the bytes to a file
        with open(f"{filename}.{image_type}", 'wb') as f:
            f.write(image_data)
        print(f"Image saved to {filename}.{image_type}")
    except Exception as e:
        print("Error saving image:", e)

def scrap_images():
    global driver
    image_types = ('fruits', 'cars', 'Machine Learning', 'guitar', 'cakes')
    create_clean_folder('images')
    try:
        for type in image_types:
            search_bar = driver.find_element(By.CSS_SELECTOR, "textarea[title='Search']")
            search_btn = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Google Search']")
            search_bar.send_keys(type)
            search_btn.click()
            time.sleep(2)
            urls = extract_images()
            print(f"Images for '{type}':")
            for url in urls:
                save_base64_image(url[1], f'images/images_{type}_{url[0]}')
            driver.back()            
    except Exception as e:
        print("Error:", e)

def main():
    try:
        init_driver("https://images.google.com/")
        if driver:
            scrap_images()
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Folder 'images' created.
Error: Message: element click intercepted: Element <button jsname="Tg7LZd" class="Tg7LZd" aria-label="Google Search" type="submit" data-ved="0ahUKEwjBnfvP2LKEAxWzk68BHeB5CzQQ4dUDCAc">...</button> is not clickable at point (787, 208). Other element would receive the click: <iframe role="presentation" frameborder="0" scrolling="no" name="callout" src="https://ogs.google.com/widget/callout?prid=19040333&amp;pgid=19037049&amp;puid=154d492226be619f&amp;cce=1&amp;dc=1&amp;origin=https%3A%2F%2Fimages.google.com&amp;cn=callout&amp;pid=1&amp;spid=2&amp;hl=en" aria-modal="true" aria-hidden="false" style="height: 100%; width: 100%; color-scheme: light;" data-iml="1708183825122"></iframe>
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF78AB37012+3522402]
	(No symbol) [0x00007FF78A758352]
	(No symbol) [0x00007FF78A605ABB]
	(No symbol) [0x00007FF78A653390]
	(No symbol) [0x00007FF78A6510D2]
	(No symbol) [0x00007FF78A64E834]
	(No symbol) [0x000

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
driver = None

def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def scrape_product_details(product):
    global driver
    try:
        search_bar = driver.find_element(By.CSS_SELECTOR, "input[placeholder='Search for Products, Brands and More']")
        search_bar.send_keys(product)
        search_bar.send_keys(Keys.RETURN)
        time.sleep(2)
        
        product_details = []
        divs = driver.find_elements(By.TAG_NAME, "div")
        products = []
        for div in divs:
            if div.get_attribute('data-tkid') != None:
                products.append(div)

        for product in products:
            try:
                product_link = product.find_element(By.CSS_SELECTOR, "a")
                product_link.send_keys(Keys.CONTROL + Keys.RETURN)  # Open link in new tab
                WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))  # Wait until new tab opens
                driver.switch_to.window(driver.window_handles[1])  # Switch to the new tab
                time.sleep(2)
                full_name = driver.find_element(By.CSS_SELECTOR, "meta[name='twitter:title']").get_attribute('content')
                details = {
                    "Brand Name": full_name.split(" ")[0],
                    "Smartphone Name": '-',
                    "Colour": '-',
                    "RAM": '-',
                    "Storage(ROM)": '-',
                    "Primary Camera": '-',
                    "Secondary Camera": '-',
                    "Display Size": '-',
                    "Battery Capacity": '-',
                    "Price": '-',
                    "Product URL": driver.current_url
                }
                driver.find_element(By.XPATH, "//button[contains(text(), 'Read More')]").click()
                spec = driver.find_element(By.XPATH, "//div[contains(text(), 'Specifications')]/following-sibling::div")
                tables = spec.find_elements(By.TAG_NAME, 'table')
                for table in tables:
                    rows = table.find_elements(By.CSS_SELECTOR, 'tr[class="row"]')
                    for row in rows:
                        cells = row.find_elements(By.TAG_NAME, 'td')
                        name = cells[0].text
                        name = name.lower()
                        if name == 'model name':
                            details['Smartphone Name'] = cells[1]

                        elif name == 'color':
                            details['Colour'] = cells[1]
                        
                        elif name == 'internal storage':
                            details['Storage(ROM)'] = cells[1]
                        
                        elif name == 'ram':
                            details['RAM'] = cells[1]
                        
                        elif name == 'primary camera':
                            details['Primary Camera'] = cells[1]
                        
                        elif name == 'secondary camera':
                            details['Secondary Camera'] = cells[1]
                        
                        elif name == 'display size':
                            details['Display Size'] = cells[1]
                        
                        elif name == 'battery capacity':
                            details['Battery Capacity'] = cells[1]
                                
                product_details.append(details)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])  # Switch back to the original tab
            except Exception as e:
                print("Error processing product:", e)
        return product_details

    except Exception as e:
        print("Error:", e)
        return []

def save_to_csv(product_details):
    df = pd.DataFrame(product_details)
    df.to_csv("amazon_products.csv", index=False)

def main():
    try:
        user_input = input("Enter the product you want to search on Flipkart: ")
        init_driver("https://www.flipkart.com")
        if driver:
            product_details = scrape_product_details(user_input)
            save_to_csv(product_details)
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Enter the product you want to search on Flipkart: Oneplus Nord, pixel 4A
Error processing product: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF78AB37012+3522402]
	(No symbol) [0x00007FF78A758352]
	(No symbol) [0x00007FF78A605ABB]
	(No symbol) [0x00007FF78A5E287C]
	(No symbol) [0x00007FF78A675D97]
	(No symbol) [0x00007FF78A68B3CF]
	(No symbol) [0x00007FF78A66EE03]
	(No symbol) [0x00007FF78A63F4D4]
	(No symbol) [0x00007FF78A6405F1]
	GetHandleVerifier [0x00007FF78AB69B9D+3730157]
	GetHandleVerifier [0x00007FF78ABBF02D+4079485]
	GetHandleVerifier [0x00007FF78ABB75D3+4048163]
	GetHandleVerifier [0x00007FF78A88A649+718233]
	(No symbol) [0x00007FF78A764A3F]
	(No symbol) [0x00007FF78A75FA94]
	(No symbol) [0x00007FF78A75FBC2]
	(No symbol) [0x00007FF78A74F2E4]
	BaseThreadInitThunk [0x00007FF923E7257D+29]
	RtlUserThreadStart [0x00007FF924C6AA58+40]

Error processing pr

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = None
def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def scrap_geo(city_name):
    global driver
    try:
        search_box = driver.find_element(By.ID, "searchboxinput")
        search_box.send_keys(city_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(5)
        return extract_coordinates_from_url(driver.current_url)
    except Exception as e:
        print("Error:", e)

def extract_coordinates_from_url(url):
    try:
        # Extract latitude and longitude from the URL
        # Example URL format: https://www.google.com/maps/place/Delhi/@28.6436846,76.7635768,10z/...
        part = url.split('@')[1]
        lat_long = part.split("/")[0].split(',')
        latitude = lat_long[0]
        longitude = lat_long[1]
        return latitude, longitude
    except ValueError:
        return None

def main():
    try:
        user_input = input("Enter the city name to retrieve details: ")
        init_driver("https://www.google.com/maps/search/")
        if driver:
            latitude, longitude = scrap_geo(user_input)
            print(f"City: {user_input}, Latitude: {latitude}, Longitude: {longitude}")
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Enter the city name to retrieve details: Lucknow
City: Lucknow, Latitude: 26.8485965, Longitude: 80.7776989


In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = None
def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def scrape_laptops():
    global driver
    try:
        time.sleep(5)
        laptops = driver.find_elements(By.CLASS_NAME, "products")
        laptop_details = []

        for laptop in laptops:
            laptop_details.append(laptop.text)

        return laptop_details
    except Exception as e:
        print("Error:", e)

def main():
    try:
        init_driver("https://www.digit.in/top-products/best-gaming-laptops-40.html")
        if driver:
            laptop_details = scrape_laptops()
            for index, detail in enumerate(laptop_details, start=1):
                print(f"Laptop {index}: {detail}")
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Laptop 1: HP Omen 17-ck2008TX 13th Gen Core i7-13700HX
Operating System:
Windows 11 Home
Display Size :
17.3
Resolution :
2560 x 1440
Processor :
13th Gen Intel Core i7-13700HX
Laptop 2: MSI GT77 Titan 12UHS-054IN 12th Gen Core i9-12900HX
Laptop 3: Lenovo Legion 5i Pro 12th Gen Core i7-12700H (82RF00E1IN)
Operating System:
Windows 11 Home
Display Size :
16
Resolution :
2560 x 1600
Processor :
12th Gen Intel Core i7-12700H
Laptop 4: ASUS ROG Strix Scar 18 G834JZ-N5041WS 13th Gen Core i9-13980HX
Laptop 5: Acer Predator Helios Neo 16 13th Gen Core i7-13700HX (PHN16-71)
Operating System:
Windows 11 Home
Display Size :
16
Resolution :
2560 x 1600
Processor :
13th Gen Intel Core i7-13700HX
Laptop 6: ASUS ROG Zephyrus G14 Ryzen 9-6900HS (GA402RJZ-L4136WS)
Laptop 7: MSI Cyborg 15 12th Gen Core i7-12650H (A12VF-205IN)


In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = None

def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def scrape_billionaires():
    global driver
    try:
        table = driver.find_element(By.CSS_SELECTOR, "div[role='rowgroup']")
        billionaires = table.find_elements(By.CSS_SELECTOR, "div[role='row']")
        billionaire_details = []

        for _, billionaire in enumerate(billionaires, start=1):
            cells = billionaire.find_elements(By.CSS_SELECTOR, 'div[role="cell"]')
            billionaire_details.append({
                "Rank": cells[0].text,
                "Name": cells[1].text,
                "Net worth": cells[2].text,
                "Age": cells[3].text,
                "Citizenship": cells[4].text,
                "Source": cells[5].text,
                "Industry": cells[6].text
            })

        return billionaire_details
    except Exception as e:
        print("Error:", e)

def main():
    try:
        init_driver("https://www.forbes.com/billionaires/")
        if driver:
            billionaire_details = scrape_billionaires()
            for _, detail in enumerate(billionaire_details, start=0):
                for key, value in detail.items():
                    print(f"{key}: {value}")
                print()
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()

Rank: 1
Name: Bernard Arnault & family
Net worth: $211 B
Age: 74
Citizenship: France
Source: LVMH
Industry: 

Rank: 2
Name: Elon Musk
Net worth: $180 B
Age: 51
Citizenship: United States
Source: Tesla, SpaceX
Industry: 

Rank: 3
Name: Jeff Bezos
Net worth: $114 B
Age: 59
Citizenship: United States
Source: Amazon
Industry: 

Rank: 4
Name: Larry Ellison
Net worth: $107 B
Age: 78
Citizenship: United States
Source: Oracle
Industry: 

Rank: 5
Name: Warren Buffett
Net worth: $106 B
Age: 92
Citizenship: United States
Source: Berkshire Hathaway
Industry: 

Rank: 6
Name: Bill Gates
Net worth: $104 B
Age: 67
Citizenship: United States
Source: Microsoft
Industry: 

Rank: 7
Name: Michael Bloomberg
Net worth: $94.5 B
Age: 81
Citizenship: United States
Source: Bloomberg LP
Industry: 

Rank: 8
Name: Carlos Slim Helu & family
Net worth: $93 B
Age: 83
Citizenship: Mexico
Source: Telecom
Industry: 

Rank: 9
Name: Mukesh Ambani
Net worth: $83.4 B
Age: 65
Citizenship: India
Source: Diversified
Industry: 


In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
driver = None

def init_driver(url):
    global driver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
    except Exception as e:
        print("Error:", e)
    return driver

def scrape_product_details(location):
    global driver
    try:
        search_bar = driver.find_element(By.TAG_NAME, "input")
        search_bar.clear()
        search_bar.send_keys(location)
        time.sleep(10)
        driver.find_element(By.CSS_SELECTOR, 'li[category="City"]').click()
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, 'button[class="btn-content medium-button icon-only"]').click()
        time.sleep(10)
        product_details = []

        # Loop through the first 3 pages (or less if fewer pages are available)
        products = driver.find_elements(By.CSS_SELECTOR, "div[class='property-info-container']")
        for product in products:
            try:
                details = {}
                try:
                    details["Hostel Name"] = product.find_element(By.CSS_SELECTOR, 'div[class="property-name"]').text
                except:
                    details["Hostel Name"] = "-"

                try:
                    details["Distance"] = product.find_element(By.CSS_SELECTOR, 'div[class="distance-description"]').text
                except:
                    details["Distance"] = "-"

                try:
                    details["Rating"] = product.find_element(By.CSS_SELECTOR, 'div[class="rating-score"]').text
                except:
                    details["Rating"] = "-"

                try:
                    details["Total Reviews"] = product.find_element(By.XPATH, '//*[@class="review"]').text
                except:
                    details["Total Reviews"] = "-"

                try:
                    details["Privates from price"] = product.find_element(By.XPATH, "//*[contains(@class, 'property-accommodation-price')][1]//child::*[contains(@class, 'current')]").text
                except:
                    details["Privates from price"] = "-"

                try:
                    details["Dorms from price"] = product.find_element(By.XPATH, "//*[contains(@class, 'property-accommodation-price')][2]//child::*[contains(@class, 'current')]").text
                except:
                    details["Dorms from price"] = "-"

                product_details.append(details)
            except Exception as e:
                print("Error processing product:", e)

        return product_details

    except Exception as e:
        print("Error:", e)
        return []

def save_to_csv(product_details):
    df = pd.DataFrame(product_details)
    df.to_csv("hostel_products.csv", index=False)

def main():
    try:
        init_driver("https://www.hostelworld.com/")
        if driver:
            product_details = scrape_product_details('london')
            save_to_csv(product_details)
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()